In [2]:
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import os
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'


2024-12-15 23:10:42.237034: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734283542.254548   63453 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734283542.260017   63453 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-15 23:10:42.279140: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Loading Pre-trained Model

In [3]:
model = tf.keras.models.load_model('emotion_recognition_model_tf.keras')

I0000 00:00:1734283544.486408   63453 gpu_process_state.cc:201] Using CUDA malloc Async allocator for GPU: 0
I0000 00:00:1734283544.486622   63453 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1265 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2050, pci bus id: 0000:01:00.0, compute capability: 8.6


In [8]:
pip install --upgrade opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 4.5 MB/s eta 0:00:00m eta 0:00:010:00:01m
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install --upgrade opencv-python opencv-python-headless


Note: you may need to restart the kernel to use updated packages.


# WebCam code

In [ ]:

path = 'haarcascade_frontalface_default.xml'
font_scale = 1.5
font = cv2.FONT_HERSHEY_PLAIN

# set the rectangle background to white
rectangle_bgr = (255,255,255)

# Make a black image
img = np.zeros((500,500))

# set some text
text = 'Some text in box'

#get the width and height of the text box
(text_width, text_height) = cv2.getTextSize(text, font, fontScale = font_scale, thickness = 1)[0]

# set the text start position
text_offset_x = 10
text_offset_y = img.shape[0] - 25

# make the coords of the box with a small padding of two pixels
box_coords = ((text_offset_x, text_offset_y), (text_offset_x + text_width + 2, text_offset_y - text_height - 2))
cv2.rectangle(img, box_coords[0], box_coords[1], rectangle_bgr, cv2.FILLED)
cv2.putText(img, text, (text_offset_x, text_offset_y), font, fontScale = font_scale, color = (0,0,0), thickness = 1)

cap = cv2.VideoCapture(0)

# Check if webcam is opened properly and correctly

if not cap.isOpened():
    cap = cv2.VideoCapture(1)
if not cap.isOpened():
    raise IOError('Cannot open webcam')

while True:
    ret,frame = cap.read()
    faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray,1.1,4)
    for x,y,w,h in faces:
        roi_gray = gray[y: y+h, x: x+w]
        roi_color = frame[y : y+h, x: x+w]
        cv2.rectangle(frame, (x,y), (x+w, y+h), (255,0,0), 2)
        faces1 = faceCascade.detectMultiScale(roi_gray)
        if len(faces1) == 0:
            print('Face not detected!!')
        else:
            for(ex, ey, ew, eh) in faces1:
                face_roi = roi_color[ey: ey+eh, ex: ex+ew] ## cropping the face 
    
    final_image = cv2.resize(face_roi, (48,48))
    final_image = np.expand_dims(final_image, axis = 0) # need fourth dimensions
    final_image = final_image/255.0 ## Normalizing
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    prediction = model.predict(final_image)

    font_scale = 1.5
    font = cv2.FONT_HERSHEY_PLAIN

    if (np.argmax(prediction) == 0):
        status = 'Angry'
        x1,y1,w1,h1 = 0,0,175,75
        # Draw black background rectangle
        cv2.rectangle(frame,(x1, x1), (x1+w1, y1+h1), (0,0,0), -1)
        # add text
        cv2.putText(frame,status, (x1 + int(w1/10), y1 + int(h1/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
        cv2.putText(frame, status, (100, 150), font, 3, (0,0,255),2, cv2.LINE_4)
        cv2.rectangle(frame, (x,y), (x+w, y+h), (0,0,255))
        
    elif (np.argmax(prediction) == 1) :
        status = 'Disgust'
        x1,y1,w1,h1 = 0,0,175,75
        # Draw black background rectangle
        cv2.rectangle(frame,(x1, x1), (x1+w1, y1+h1), (0,0,0), -1)
        # add text
        cv2.putText(frame,status, (x1 + int(w1/10), y1 + int(h1/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
        cv2.putText(frame, status, (100, 150), font, 3, (0,0,255),2, cv2.LINE_4)
        cv2.rectangle(frame, (x,y), (x+w, y+h), (0,0,255))
    
    elif (np.argmax(prediction) == 2) :
        status = 'Fear'
        x1,y1,w1,h1 = 0,0,175,75
        # Draw black background rectangle
        cv2.rectangle(frame,(x1, x1), (x1+w1, y1+h1), (0,0,0), -1)
        # add text
        cv2.putText(frame,status, (x1 + int(w1/10), y1 + int(h1/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
        cv2.putText(frame, status, (100, 150), font, 3, (0,0,255),2, cv2.LINE_4)
        cv2.rectangle(frame, (x,y), (x+w, y+h), (0,0,255)) 
    
    elif (np.argmax(prediction) == 3) :
        status = 'Happy'
        x1,y1,w1,h1 = 0,0,175,75
        # Draw black background rectangle
        cv2.rectangle(frame,(x1, x1), (x1+w1, y1+h1), (0,0,0), -1)
        # add text
        cv2.putText(frame,status, (x1 + int(w1/10), y1 + int(h1/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
        cv2.putText(frame, status, (100, 150), font, 3, (0,0,255),2, cv2.LINE_4)
        cv2.rectangle(frame, (x,y), (x+w, y+h), (0,0,255)) 
    
    elif (np.argmax(prediction) == 4) :
        status = 'Sad'
        x1,y1,w1,h1 = 0,0,175,75
        # Draw black background rectangle
        cv2.rectangle(frame,(x1, x1), (x1+w1, y1+h1), (0,0,0), -1)
        # add text
        cv2.putText(frame,status, (x1 + int(w1/10), y1 + int(h1/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
        cv2.putText(frame, status, (100, 150), font, 3, (0,0,255),2, cv2.LINE_4)
        cv2.rectangle(frame, (x,y), (x+w, y+h), (0,0,255)) 
   
    elif (np.argmax(prediction) == 5) :
        status = 'Surprise'
        x1,y1,w1,h1 = 0,0,175,75
        # Draw black background rectangle
        cv2.rectangle(frame,(x1, x1), (x1+w1, y1+h1), (0,0,0), -1)
        # add text
        cv2.putText(frame,status, (x1 + int(w1/10), y1 + int(h1/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
        cv2.putText(frame, status, (100, 150), font, 3, (0,0,255),2, cv2.LINE_4)
        cv2.rectangle(frame, (x,y), (x+w, y+h), (0,0,255)) 
    
    elif (np.argmax(prediction) == 6) :
        status = 'Neutral'
        x1,y1,w1,h1 = 0,0,175,75
        # Draw black background rectangle
        cv2.rectangle(frame,(x1, x1), (x1+w1, y1+h1), (0,0,0), -1)
        # add text
        cv2.putText(frame,status, (x1 + int(w1/10), y1 + int(h1/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
        cv2.putText(frame, status, (100, 150), font, 3, (0,0,255),2, cv2.LINE_4)
        cv2.rectangle(frame, (x,y), (x+w, y+h), (0,0,255)) 
     
    
     
    cv2.imshow('Face and Emotion Recognition', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined